# **INTRODUCTION**

A l'heure actuel, le monde des échecs professionnels connait une vrai crise : des parties de tournois majeurs pas retransmises direct live, scandale de triches, Magnus Carlsen viré en plein championnat du monde pour être venue en "tenue non-conforme" (un pantalon en jean...). Ces incidents ont pour conséquence une perte d'interet du public pour la FIDE (Fédération International D'Echecs) et pour le format de parti longue. Mais s'il y a bien une chose qui est sur une bonne dynamique dans les échecs, c'est bien l'IA. Cela fait bien des années que l'intelligence artificielle dépasse ce qui est phyiquement possible a faire pour l'humain dans ce domaine.

Parmis tout les moteurs d'échecs ayant existés (Deep-Blue, etc...), le requin de cette catégorie, c'est Stockfish. C'est un moteur d'échecs open source créé en 2008 par Tord Romstad, Marco Costalba et Joona Kiiski. Développé et amélioré par une communauté active, il est rapidement devenu l’un des moteurs d’échecs les plus puissants au monde. Stockfish fonctionne en combinant des algorithmes de recherche avancés, comme l’élagage alpha-bêta et le Deep Iterative Deepening, avec une fonction d’évaluation capable d’analyser les positions selon divers critères : avantage matériel, structure des pions, sécurité du roi, etc. Depuis 2020, il intègre NNUE, un réseau de neurones optimisé pour les processeurs, qui améliore la précision de ses évaluations. Aujourd'hui, Stockfish est utilisé pour analyser des parties, entraîner des joueurs, organiser des tournois en ligne et développer des projets d’intelligence artificielle. Gratuit et open source, il est devenu incontournable pour les amateurs et professionnels des échecs.

Pour en arrivé jusque la, Stockfish s'appuie dans la combinaisons de multitude d'algorithmes, de technologies, méthodes analyses et entrainement depuis plus de 15 ans. Le "jeu des rois" étant devenue le "jeu des robots", nous allons faire de même. Notre objectif dans ce projet, est alors de créer une l'IA joueuse d'échecla plus performante possible, à l'aide un réseaux de neurone.

---

# **1. CHOIX DU JEU DE DONNEE**

Une des meilleure source donee en terme d'echec est Lichess (abréviation de "Libre Chess"), vest une plateforme gratuite et open-source dédiée aux échecs en ligne. Elle permet aux joueurs de tous niveaux de jouer, apprendre et s'améliorer aux échecs via navigatuer web ou une application mobile. Plusieurs banques de données sont disponibles en libre accès sur leur site internet, parmis elles, deux sont de bonnes candidates pour notre sujet :
- **Standars Chess** : Cette basse de donnée regroupe toutes les parties classées qui ont été joué sur lichess, chaque mois, depuis sa création, soit 6 298 645 464 parties standard.
- **Evaluation"** : 173 866 932 positions d'échecs différentes évaluées avec Stockfish. Produites par et pour le tableau d'analyse de Lichess, utilisant différentes versions de Stockfish dans les navigateurs des utilisateurs.

BLABLABLA a completer et corriger : Grace a notre reflexion personnelle et a l'avis de ChatGPT, nous avons vue que le choix de prendre les données "Evaluation" est notre meilleur choix : 
stockifsh est + fort que nimporte quel joueur donc meilleurs données + même parmis toutes les parties il faut prendre celles avec les top joueurs et même eux peuvent faire de la merde parfois donc pas 100 fiables. Stockfish que des positions différentes et plusieurs evaluations différentes pour chacune donc c'est hyper pertinent pas de données a jeter.

Même si cela revient a utiliser une IA pour entrainer notre IA, Stockfish lui même s'appuyer sur le moteur "Glaurung", il vaut mieux mettre toute les chances de notre coté et prendre les meilleurs données a notre dispositon.

# **2. EXPLORATION ET TRAITEMENT DES DONNEES**

## 2.A Extraction des données brutes

Extraction des données depuis le fichier JSONL d'origine qui contient 170M positions différentes, chacune avec plusieurs évaluations stockfish . Nous limitons alors la taille de notre base de donnée a **500 000 positions**.

In [8]:
file_path = 'data\lichess_db_eval.jsonl'
data_raw = extract_data(file_path, max_rows=5000)
data_raw.head(5)

,fen,knodes,depth,cp,mate,line
0,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,200973,39,58.0,NaN,f7g7 e6e2 h8d8 e2d2 b7b5 c4e6 g7f6 e6b3 a6a5 a2a3
1,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,62.0,NaN,f7g7 e6e2 b7b5 c4b3 h8d8 e2d2 a6a5 a2a3 g7f6 d1e1
2,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,151.0,NaN,h8d8 d1e1 a6a5 a2a3 b7b5 c4a2 c6d7 e6e7 f7g6 e1f2
3,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,59730,31,64.0,NaN,f7g7 e6e2 g7g6 d1c2 h8d8 e2d2 g6f6 a2a3 b7b5 c4a2
4,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,59730,31,134.0,NaN,h8d8 d1e1 a6a5 a2a3 b7b5 c4b3 a5a4 b3a2 c6d7 e6h6


La base de donnée est composée de **2 330 098 évaluations différentes** et **5 variables** : 

| Variable            | Description                                                                                                        |
|-----------------|--------------------------------------------------------------------------------------------------------------------|
| **knodes**      | Nombre de milliers de nœuds analysés par stockfish pendant la recherche de la position.                   |
| **depth**       | Profondeur de la recherche, indiquant le nombre de coups (ou niveaux) explorés par le moteur.                     |
| **cp**          | Évaluation en centipions, représentant un avantage matériel en fonction du côté actif (positif pour Blancs, négatif pour Noirs). |
| **mate**        | Évaluation de mat. Si une valeur est donnée, elle indique le nombre de coups restants avant que le mat soit atteint. |
| **line**        | Ligne (parfois appelée variante) désigne une séquence de coups qui découle d'une position donnée. Elle représente une possible continuation du jeu |

## 2.B Nettoyage des données

Stockfish utilie différents parametre dans son algorythme, notamment le **nombre de noeuf (knodes)** et le la **profondeur (depth)**. En fonction de ces différents parametres, stockfish calcul le **centipions (cp)** et **l'évaluation du mate (mate)**.  

Après ces évaluations, stockfish peut alors parfois prédire des coups différents pour une même position, on appelle cela des **variantes**. Ces coups sont différents mais ont un impacte similaire ou quasi-similaire sur l'avantage qu'il donne a celui qui le joue (en terme d'evaluation de la position). 

Certains joueurs vont être plus a l'aise dans une variante plûtot qu'une autre en fonction de leurs styles de jeu ou de leurs connaissances, mais pour une IA cela n'as pas d'importance ! Toutefois, notre source de donnée nous propose plusieurs analyse de stockfish pour chaques position, apportant son lot de variante.

Nous pouvons alors utiliser 2 méthodes pour garder la meilleur variante.

### **Méthode N°1 : Analyse des Parametres et de l'Evaluation**
   1. **Profondeur de recherche (depth)** : Plus la profondeur est grande, plus l'analyse est précise. Priorisations des évaluations avec la profondeur maximale.

   2. **Centipions (cp) ou mat (mate)** : Si une évaluation donne un **mate**, elle est prioritaire, car un mat forcé est absolu. En absence de **mate**, choix d'évaluation avec la valeur **cp** la plus élevée pour le joueur actif

   3. **Nombre de nœuds analysés (knodes)** : Si plusieurs évaluations ont la même profondeur, celle ayant exploré le plus grand nombre de nœuds est théoriquement plus fiable.

   4. **En cas d'égalité parfaite**: En cas d’égalité sur les autres critères, utilisez une évaluation arbitraire.

### **Méthode N°2 : Coup le Plus Populaire** 
blablablabla


- Création d'une variable `best_move_m1`, qui extrait le prochain coup a jouer, c'est a dire le premier coup de la ligne.

In [9]:
data_V1 = extract_next_move(data_raw)
data_V1.head(3)

,fen,knodes,depth,cp,mate,line,best_move_m1
0,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,200973,39,58.0,NaN,f7g7 e6e2 h8d8 e2d2 b7b5 c4e6 g7f6 e6b3 a6a5 a2a3,f7g7
1,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,62.0,NaN,f7g7 e6e2 b7b5 c4b3 h8d8 e2d2 a6a5 a2a3 g7f6 d1e1,f7g7
2,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,151.0,NaN,h8d8 d1e1 a6a5 a2a3 b7b5 c4a2 c6d7 e6e7 f7g6 e1f2,h8d8


- Création d'une variable `best_move_m2` avec le coup le plus populaire par position en suivant la Méthode N°2

In [10]:
data_V2 = most_popular_predict_V1(data_V1)
data_V2.head(3)

,fen,knodes,depth,cp,mate,line,best_move_m1,best_move_m2
0,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,200973,39,58.0,NaN,f7g7 e6e2 h8d8 e2d2 b7b5 c4e6 g7f6 e6b3 a6a5 a2a3,f7g7,f7g7
1,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,62.0,NaN,f7g7 e6e2 b7b5 c4b3 h8d8 e2d2 a6a5 a2a3 g7f6 d1e1,f7g7,f7g7
2,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,151.0,NaN,h8d8 d1e1 a6a5 a2a3 b7b5 c4a2 c6d7 e6e7 f7g6 e1f2,h8d8,f7g7


- Filtrage des lignes pour ne garder qu'une evaluation par position selon la méthode N°1.

In [11]:
data_V3 = filter_best_predict(data_V2)
data_V3.head(3)

,fen,knodes,depth,cp,mate,line,best_move_m1,best_move_m2
0,1K6/1P4p1/2n4k/p6p/8/8/8/8 w - -,159161,29,NaN,-12.0,b8a8 a5a4 b7b8b c6b8 a8b8 a4a3 b8c8 a3a2 c8d7 ...,b8a8,b8c8
1,1K6/1P4p1/7k/p3n2p/8/8/8/8 b - -,186187,25,460.0,NaN,e5c6 b8c7 c6b4 c7b6 b4d5 b6c6 d5b4 c6b5 h6g5 b...,e5c6,e5c6
2,1K6/2Pk4/b7/5ppp/8/p5P1/4PP1P/B7 w - -,1472,39,0.0,NaN,e2e3 a3a2 a1c3 a6c8 f2f3 g5g4 f3g4 h5g4 c3d4 c8a6,e2e3,e2e3


- Suppression des variables inutiles. Les variables `knodes`, `depth`, `cp`, `mate` et `line` ne sont plus utile pour la suite du projet

In [12]:
data_cleaned = drop_usuless_columns(data_V3)
data_cleaned.head(5)

,fen,best_move_m1,best_move_m2
0,1K6/1P4p1/2n4k/p6p/8/8/8/8 w - -,b8a8,b8c8
1,1K6/1P4p1/7k/p3n2p/8/8/8/8 b - -,e5c6,e5c6
2,1K6/2Pk4/b7/5ppp/8/p5P1/4PP1P/B7 w - -,e2e3,e2e3
3,1Kb2n2/2P1k3/2B5/4Bppp/8/p5P1/4PP1P/8 b - -,f8d7,c8e6
4,1Kb5/2PBk3/8/4Bppp/8/p5P1/4PP1P/8 b - -,c8d7,e7d7


- Sauvegarde des données netoyées en CSV

In [13]:
data_cleaned.to_csv('data/data_cleaned.csv')

---

# **3. MODELISATION**

## 3.A Transformation finale des données :

blablablabla

- Chargement des Données

In [2]:
data = pd.read_csv('data/data_cleaned.csv' )

X = data['fen']
y = data['best_move_m1']

- Traduction des données sous une forme interpretable par le reseau de neurone.
  - **Position (Fen)** -> Matrice Numpy
  - **Coup a joué (UCI)** -> Encodage de chaque coup distincts (sous forme d'entier)

In [3]:
X = X.apply(fen_to_matrix)
X = np.array(X.tolist())

# Encode les mouvements et conversion en catégories
y, move_to_int = encode_moves(y)
y = to_categorical(y, num_classes=len(move_to_int))


# Sauvegarde de move_to_int dans un fichier JSON
with open('Models/move_int_dico.json', 'w') as file:
    json.dump(move_to_int, file)


KeyboardInterrupt



## 3.B Définition et entrainement des différents modèles

### Modèle N°1 :

### Modèle N°2 :

- Definition du Modèle

In [15]:
model_1 = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(13, 8, 8)),
    BatchNormalization(),  # Ajout de BatchNormalization pour stabiliser l'entraînement
    MaxPooling2D(pool_size=(2, 2)),  # Réduction de la dimension spatiale
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dropout(0.4),  # Dropout réduit à 40% ici pour limiter le surapprentissage
    Dense(256, activation='relu'),
    Dropout(0.3),  # Dropout progressif pour éviter un réseau trop complexe
    Dense(len(move_to_int), activation='softmax')  # Couche de sortie
])

# Optimiseur avec un learning rate ajusté
model_1.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

model_1.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 11, 6, 64)      │         4,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 11, 6, 64)      │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 5, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 3, 1, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 3, 1, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 1, 0, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 0)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 0)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1927)           │       495,239 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 574,791 (2.19 MB)

 Trainable params: 574,407 (2.19 MB)

 Non-trainable params: 384 (1.50 KB)

- Entrainement du Modèle

In [ ]:
model_1.fit(X, y, epochs=25, validation_split=0.1, batch_size=64)

### **Modèle N°3** :

# **4. EVALUATION**